In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
"""
Purpose: Want to calcultae the conversion rate
between of 2 different groups of cells
onto one another (and the compartment conversion rate).
The groups of cells will be determined as downstream
targets from 2 inhibitory cells

Pseudocode: 
1) Load the data (previously generated by NEURD 
decomposition and proximity pipeline
2) Restrict dataframe to 2 inhibitory presyn neurons
3) Get the targets for each presyn neuron

4) For each targets group
a. Restrict the dataframe to presyns from this group
b. Restrict the dataframe to postsyns in the other group
c. Calculate the compartment conversion rate

"""
print()

In [3]:
from python_tools import pandas_utils as pu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
from neurd import proximity_analysis_utils as pau

In [5]:
verbose = True

# Load the data

In [6]:
#df = pu.csv_to_df("./large_inh_and_downstream_23p_prox_table.csv")
df = pu.csv_to_df("/neurd_packages/neuron_mesh_tools/Auto_Proofreading/Minnie65_vs_H01/Connectome/Cell_Type_Motif_Search/large_inh_and_downstream_23p_prox_table.csv")
df

,presyn,postsyn,n_synapses,postsyn_compartment,presyn_cell_type,postsyn_cell_type,proximity_dist,n_spine_post,n_synapse_post,n_synapse_pre_proof,...,presyn_width,prox_id,proximity_dist_non_adjusted,synapse_id,synapse_id_dist,segment_id,split_index,segment_id_post,split_index_post,gnn_embed_method
0,864691134884836602_0,864691135155503716_0,0,basal,23P,23P,1047.8600,6,6,0,...,81.9550,1,1421.870,NaN,NaN,864691134884836602,0,864691135155503716,0,a5699b4c
1,864691134884836602_0,864691135155503716_0,0,apical,23P,23P,1440.9700,3,4,0,...,70.6595,2,1727.000,NaN,NaN,864691134884836602,0,864691135155503716,0,a5699b4c
2,864691134884836602_0,864691135155550308_0,0,basal,23P,23P,-66.1744,4,6,0,...,66.0733,1,450.249,NaN,NaN,864691134884836602,0,864691135155550308,0,a5699b4c
3,864691134884836602_0,864691135210063040_0,0,basal,23P,23P,1542.4800,9,12,0,...,67.5147,1,1982.250,NaN,NaN,864691134884836602,0,864691135210063040,0,a5699b4c
4,864691134884836602_0,864691135256202927_0,0,basal,23P,23P,-30.1768,7,9,0,...,100.0710,1,572.418,NaN,NaN,864691134884836602,0,864691135256202927,0,a5699b4c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37642,864691137054006902_0,864691136925807178_0,0,basal,23P,23P,1305.4100,9,8,0,...,59.3964,4,1705.370,NaN,NaN,864691137054006902,0,864691136925807178,0,a5699b4c
37643,864691137054006902_0,864691136925807178_0,0,basal,23P,23P,2159.5100,7,5,0,...,89.3006,5,2529.170,NaN,NaN,864691137054006902,0,864691136925807178,0,a5699b4c
37644,864691137054006902_0,864691136965766094_0,0,basal,23P,23P,1988.5800,2,6,1,...,65.7788,2,2375.280,NaN,NaN,864691137054006902,0,864691136965766094,0,a5699b4c
37645,864691137054006902_0,864691136973716252_0,0,basal,23P,23P,1137.4800,1,6,1,...,65.7568,1,1341.720,NaN,NaN,864691137054006902,0,864691136973716252,0,a5699b4c


# Determining two target groups

In [7]:
import numpy as np

n1 = "864691135939859457_0"
n2 = "864691136041433430_0"

df1 = df.query(f"presyn == '{n1}'")
df2 = df.query(f"presyn == '{n2}'")

n1_targets = df1['postsyn'].unique()
n2_targets = df2['postsyn'].unique()

n1_targets_mut_exc = np.setdiff1d(n1_targets,
                                 n2_targets)
n2_targets_mut_exc = np.setdiff1d(n2_targets,
                                 n1_targets)
targets_intersection = np.intersect1d(n1_targets,n2_targets)

if verbose:
    print(f"proximity df neuron 1 = {len(df1)}")
    print(f"n1_targets = {len(n1_targets)}")
    print(f"n1_targets_mut_exc = {len(n1_targets_mut_exc)}")
    print(f"----")
    print(f"proximity df neuron 2 = {len(df2)}")
    print(f"n2_targets = {len(n2_targets)}")
    print(f"n2_targets_mut_exc = {len(n2_targets_mut_exc)}")
    print(f"--- ")
    print(f"targets_intersection = {len(targets_intersection)}")

proximity df neuron 1 = 561
n1_targets = 242
n1_targets_mut_exc = 104
----
proximity df neuron 2 = 401
n2_targets = 201
n2_targets_mut_exc = 63
--- 
targets_intersection = 138


In [8]:
group_1 = n1_targets_mut_exc
group_2 = n2_targets_mut_exc

# Calculating the compartment conversion rate

In [15]:
def inter_group_conversion_rate(
    df,
    presyns,
    postsyns,
    ):
    
    """
    Purpose: 
    """
    df_restr = pu.restrict_df_from_list(
        df,
        restrictions=[
        f"presyn in {list(presyns)}",
        f"postsyn in {list(postsyns)}",
        ]
    ).reset_index(drop=True)
    
    return pau.conversion_df(
        df_restr,
        verbose = True
    )
  

In [16]:
inter_group_conversion_rate(
    df,
    group_1,
    group_2
)

# of pairs = 5 (time = 0.021)


,postsyn_compartment,n_synapses,n_proximity,proximity_dist,presyn_width,conversion
0,apical,0,45,1337.514340,77.244776,0.000000
1,apical_shaft,1,8,1329.576875,82.255700,0.125000
2,basal,4,456,1359.845852,80.046344,0.008772
3,oblique,0,33,1180.756097,80.442897,0.000000
4,soma,0,12,1080.214825,68.665650,0.000000


In [17]:
inter_group_conversion_rate(
    df,
    group_2,
    group_1
)

# of pairs = 5 (time = 0.047)


,postsyn_compartment,n_synapses,n_proximity,proximity_dist,presyn_width,conversion
0,apical,1,25,1092.623184,78.858436,0.040000
1,apical_shaft,0,6,1600.986667,79.030550,0.000000
2,basal,3,359,1339.919363,80.365336,0.008357
3,oblique,0,43,1124.917237,81.666716,0.000000
4,soma,0,10,1175.551000,74.712810,0.000000
